In [16]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Deep Learning libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU, SimpleRNN
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("All libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

All libraries imported successfully!
TensorFlow version: 2.19.0
NumPy version: 2.0.2
Pandas version: 2.2.2


In [17]:
# Kaggle paths
BASE_DIR = "/kaggle/working"
DATA_PATH = f"{BASE_DIR}/data"
MODELS_PATH = f"{BASE_DIR}/models"
OUTPUTS_PATH = f"{BASE_DIR}/outputs"

# Create directories
for path in [DATA_PATH, MODELS_PATH, OUTPUTS_PATH]:
    os.makedirs(path, exist_ok=True)

print("Project directories created in Kaggle")
print(f"Data directory: {DATA_PATH}")
print(f"Models directory: {MODELS_PATH}")
print(f"Outputs directory: {OUTPUTS_PATH}")

# Competition input path (CHANGE competition name if needed)
COMPETITION_PATH = "/kaggle/input/formative-1-time-series-forecasting-january-2026"

train_path = f"{COMPETITION_PATH}/train.csv"
test_path = f"{COMPETITION_PATH}/test.csv"
sample_path = f"{COMPETITION_PATH}/sample_submission.csv"

print("\nFile existence check:")
print(f"train.csv exists: {os.path.exists(train_path)}")
print(f"test.csv exists: {os.path.exists(test_path)}")
print(f"sample_submission.csv exists: {os.path.exists(sample_path)}")

Project directories created in Kaggle
Data directory: /kaggle/working/data
Models directory: /kaggle/working/models
Outputs directory: /kaggle/working/outputs

File existence check:
train.csv exists: True
test.csv exists: True
sample_submission.csv exists: False
